In [15]:
import onnx
import onnxruntime
from utils import loader
import numpy as np

onnx_model = onnx.load("FCN_outputModel.onnx")
onnx.checker.check_model(onnx_model)



In [16]:
#### Adding STEP dataset###############

#base_path = os.path.dirname(os.path.realpath(__file__))
base_path = ''
ftype = ''   ### to change for differing subparts of step dataset
coords = 3
joints = 16
cycles = 1
test_size = 0.1
data, labels,\
    data_train, labels_train,\
    data_test, labels_test = loader.load_data(base_path, ftype, coords, joints,
                                              cycles=cycles, test_size=test_size)
data_train_new = np.swapaxes(data_train,1,2)
data_test_new = np.swapaxes(data_test,1,2)




In [29]:
print(data_train_new[6].shape)

(48, 75)


In [32]:
#### Running an inference using onnx model######
ort_session = onnxruntime.InferenceSession("FCN_outputModel.onnx")
data_to_test = data_train_new[25].astype(np.float32)
data_to_test = np.expand_dims(data_to_test,0)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() 

ort_inputs = {ort_session.get_inputs()[0].name: data_to_test}
ort_outs = ort_session.run(None, ort_inputs)
#np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
max = float('-inf')
max_index = -1
for i in range(0, len(ort_outs[0][0])):       
   if(ort_outs[0][0][i] > max):    
       max = ort_outs[0][0][i]
       max_index = i
print(max_index)

1
